In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!nvidia-smi

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CUDNN_USE_AUTOTUNE'] = '0'

In [4]:
import tensorflow as tf
config = tf.ConfigProto(intra_op_parallelism_threads=0, 
                        inter_op_parallelism_threads=0, 
                        allow_soft_placement=True)
session = tf.Session(config=config)
tf.keras.backend.set_session(session)

In [5]:
import random
import numpy as np

seed = 11

np.random.seed(seed)
random.seed(seed)
tf.random.set_random_seed(seed)

In [6]:
from fewshot.data_provider.dataset import Dataset
from fewshot.backbones import ConvNet

from fewshot.algorithms.backbone_train import simple_one_layer_cross_entropy_train
from fewshot.algorithms.fewshot_test import baseline_fewshot_test
from fewshot.algorithms.fewshot_models import BaselineFewShotModel

import tqdm
from scipy import stats

## Initialize

Data parameters:

In [7]:
img_width = 84
img_height = 84
img_depth = 3

In [10]:
dataset_root = "../../../data/"
dataset_name = 'cub'
csv_name = "data_84x84.csv"

base_num_classes = 100
novel_num_classes = 50
val_num_classes = 50  # not used in baseline algo

Backbone training parameters:

In [11]:
# backbone training
backbone_training_batch_size = 16
backbone_training_epochs = 20

backbone_generator_args={
    "flip_prob": 0.5,
    "color_jitter_prob": 1.0,
    "hue_range": (0.6, 1.4),
    "saturation_range": (0.6, 1.4),
    "value_range": (0.6, 1.4)
}

Few-shot training parameters:

In [12]:
# fewshot training and testing
n_way = 5
k_shot = 5
fewshot_batch_size = 4
support_epochs = 100
query_size = 16

support_generator_args={
    "flip_prob": 0.5,
    "color_jitter_prob": 1.0,
    "hue_range": (0.6, 1.4),
    "saturation_range": (0.6, 1.4),
    "value_range": (0.6, 1.4)
}

n_episodes = 10

In [13]:
checkpoint_dir = '../fewshot/checkpoints'
log_dir = '../fewshot/logs'

## Data

Create dataset:

In [14]:
dataset_dir = os.path.join(dataset_root, dataset_name)
dataset = Dataset(dataset_dir=dataset_dir, 
                  csv_name=csv_name, 
                  image_size=(img_width, img_height))

Create backbone dataset:

In [15]:
backbone_dataset, val_fewshot_dataset = dataset.split_by_classes(train_size=base_num_classes,
                                                                 random_state=seed)

Split by classes with train size = 100 (seed = 11)
Train classes: 100
Test classes: 100
Train data: 3043 samples
Test data:  2990 samples


/mnt/disk/venv/dpy3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Create few-shot dataset:

In [16]:
val_dataset, fewshot_dataset = val_fewshot_dataset.split_by_classes(train_size=val_num_classes,
                                                                    random_state=seed)

Split by classes with train size = 50 (seed = 11)
Train classes: 50
Test classes: 50
Train data: 1510 samples
Test data:  1480 samples


## Backbone

In [18]:
model_name = 'baseline'

Create model:

In [19]:
backbone = ConvNet(input_size=(img_width, img_height, img_depth))
backbone.set_trainable(True)

Instructions for updating:
Colocations handled automatically by placer.


Train model:

In [20]:
backbone_optimizer = tf.keras.optimizers.Adam(lr=1e-3)

In [21]:
simple_one_layer_cross_entropy_train(
    backbone,
    backbone_dataset.get_batch_generator(batch_size=backbone_training_batch_size,
                                         shuffle=True,
                                         generator_args=backbone_generator_args),
    epochs=backbone_training_epochs,
    optimizer=backbone_optimizer,
    model_name=model_name,
    checkpoint_dir=checkpoint_dir,
    period=10,
    tensorboard=True,
    log_dir=log_dir,
    use_multiprocessing=True,
    workers=4,
    verbose=2
)

Epoch 1/20
 - 9s - loss: 5.1533 - acc: 0.0233
Epoch 2/20
 - 6s - loss: 4.4799 - acc: 0.0424
 - 9s - loss: 5.1533 - acc: 0.0233
Epoch 3/20
Epoch 2/20
 - 6s - loss: 4.1929 - acc: 0.0634
Epoch 4/20
 - 6s - loss: 3.9895 - acc: 0.0920
Epoch 5/20
 - 6s - loss: 3.7819 - acc: 0.1222
Epoch 6/20
 - 6s - loss: 3.5958 - acc: 0.1531
Epoch 7/20
 - 7s - loss: 3.3546 - acc: 0.1968
Epoch 8/20
 - 6s - loss: 3.1754 - acc: 0.2323
Epoch 9/20
 - 6s - loss: 2.9925 - acc: 0.2636
Epoch 10/20
 - 7s - loss: 2.7388 - acc: 0.3221
Epoch 11/20
 - 6s - loss: 2.5349 - acc: 0.3671
Epoch 12/20
 - 6s - loss: 2.3151 - acc: 0.4006
Epoch 13/20
 - 6s - loss: 2.1221 - acc: 0.4561
Epoch 14/20
 - 6s - loss: 1.9120 - acc: 0.4982
Epoch 15/20
 - 6s - loss: 1.6995 - acc: 0.5580
Epoch 16/20
 - 6s - loss: 1.5478 - acc: 0.5945
Epoch 17/20
 - 6s - loss: 1.3722 - acc: 0.6283
Epoch 18/20
 - 6s - loss: 1.2731 - acc: 0.6687
Epoch 19/20
 - 6s - loss: 1.0692 - acc: 0.7226
Epoch 20/20
 - 6s - loss: 0.9334 - acc: 0.7667


## Few-shot

In [22]:
episode_generator = fewshot_dataset.get_fewshot_generator(n_way=n_way, 
                                                          k_shot=k_shot, 
                                                          query_size=query_size,
                                                          support_generator_args=support_generator_args)

Train few-shot model:

In [23]:
fewshot_optimizer = tf.keras.optimizers.Adam(lr=1e-3) 

In [24]:
fewshot_model = BaselineFewShotModel(backbone, n_way)
accuracies = baseline_fewshot_test(model=fewshot_model,
                                   generator=episode_generator, 
                                   optimizer=fewshot_optimizer,
                                   batch_size=fewshot_batch_size,
                                   support_epochs=support_epochs,
                                   n_episodes=n_episodes,
                                   model_name='baseline-fewshot',
                                   tensorboard=True,
                                   log_dir=log_dir,
                                   period=10)

  0%|          | 0/10 [00:00<?, ?it/s]

Instructions for updating:
Use tf.cast instead.


Average acc: 46.00%: 100%|██████████| 10/10 [01:06<00:00,  7.06s/it]


## Report

Calculate 95% confidence interval:

In [25]:
acc_int = stats.t.interval(0.95, len(accuracies) - 1, loc=np.mean(accuracies), scale=stats.sem(accuracies))
print("5-shot test metric: {}% +- {}%".format(round(np.mean(accuracies) * 100, 2),
                                              round((np.mean(acc_int) - acc_int[0]) * 100, 2)))

5-shot test metric: 46.0% +- 6.49%
